# Indirect TSQR


**Input**: Matrix $A \in \mathbb{R}^{m \times n}$ with $m \gg n$.  

The indirect TSQR method avoids explicitly assembling the final $Q$ matrix block-by-block. The idea is to first compute a stable global $R$ factor first, and then derives $Q$ implicitly from it.  

---

### 1) First step (local QR factorizations)  
- The matrix $A$ is divided into $p$ row blocks:  

$$
A = \begin{bmatrix} A_1^T & A_2^T & \cdots & A_p^T \end{bmatrix}^T,
\quad A_j \in \mathbb{R}^{m_j \times n}.
$$  

- Each block is used locally to factor the small R_j blocks:  

$$
A_j = Q_j^{(1)} R_j, 
\quad Q_j^{(1)} \in \mathbb{R}^{m_j \times n}, \; R_j \in \mathbb{R}^{n \times n}.
$$  

  
- The $Q_j^{(1)}$ matrices obtained are discarded and only the small $R_j$ are passed along.  



### 2) Second step (global QR reduction)  
- The local triangular matrices are stacked vertically:  

$$
R_{\text{stack}} = 
\begin{bmatrix} 
R_1 \\ R_2 \\ \vdots \\ R_p 
\end{bmatrix}
\in \mathbb{R}^{pn \times n}.
$$  

- To obtain the **final global $R$** it is necessary to perform a second QR factorization:  

$$
R_{\text{stack}} = \tilde{Q} \, R, 
\quad \tilde{Q} \in \mathbb{R}^{pn \times n}, \; R \in \mathbb{R}^{n \times n}.
$$  

- Unlike other methods, the $Q_j^{(1)}$ blocks are not explicitily multiplied with pieces of $\tilde{Q}$ to assemble the final $Q$, instead they are discarded.  



### 3) Recovering $Q$ indirectly  
- By construction, $A = Q R$.  
- Since $R$ is already available, $Q$ can be obtained as:  

$$
Q = A R^{-1}.
$$  

- This avoids explicitly combining the intermediate $Q_j^{(1)}$ and $\tilde{Q}$ matrices.  
- Instead, a final *map* step applies the small matrix $R^{-1}$ (size $n \times n$) to each row block of $A$, yielding the blocks of $Q$ on the fly.  

---

The optimization idea is that only  the small $R_j$ factors are passed between workers, never the large $Q_j^{(1)}$. The tradeoff is that it  requires access to the full $A$ again to compute $Q$, which may be costly for very large datasets, but avoids storing intermediate $Q_j$.
On the other hand the two-level QR decomposition ensures orthogonality and therefore an improved numerical stability.  
  


In [ ]:
import dask.array as da
import dask
import numpy as np
import time
from scipy.linalg import solve_triangular 
from sklearn.datasets import fetch_california_housing


In [ ]:
# CLUSTER DEPLOYMENT ON CLOUDVENETO
from dask.distributed import Client, SSHCluster

cluster = SSHCluster(
    ["10.67.22.154", "10.67.22.216", "10.67.22.116", "10.67.22.113"],
    connect_options={"known_hosts": None},
    remote_python="/home/ubuntu/miniconda3/bin/python",
    scheduler_options={"port": 8786, "dashboard_address": ":8797"},
    worker_options={
        "n_workers": 4,       # N. of processess per VM. CloudVeneto's large VM offers 4-core CPU, but for now we only spawn 1 process per VM
        "nthreads": 1      # N. of threads per process
    }
)

client = Client(cluster)


In [ ]:
# check if everything went smoothly
print(client)
print(cluster)


<Client: 'tcp://127.0.0.1:38527' processes=4 threads=4, memory=5.79 GiB>
LocalCluster(102e741e, 'tcp://127.0.0.1:38527', workers=4, threads=4, memory=5.79 GiB)


In [ ]:
X_cached = X_da.rechunk((1_000_000, -1)).persist()

X_da


dask.array<array, shape=(20640, 8), dtype=float64, chunksize=(6880, 8), chunktype=numpy.ndarray>


## Variants of the Indirect TSQR Method
In the following, different versions of the **Indirect TSQR** algorithm are presented.  
The main difference across these implementations lies in how the global $R$ factor is handled: computed, persisted across workers, or kept as a delayed/Dask object.  

---

• **Serial version of the Indirect TSQR**

This approach showcases the **basic formulation** of the indirect method using only NumPy. It has some serious limitations due to the lack of parallelizzation as everything runs on a single core The memory usage scales with the dataset size, making this approach infeasible for very large datasets (e.g. HIGGS) and only suitable for local analysis.

---
• **Parallel / Dask approach (version 1)**

This version introduces Dask for parallelism. Each block of $A$ is processed in parallel using the function:
```python
R_blocks = X_da.map_blocks(compute_R, dtype=X_da.dtype, chunks=(n_cols, n_cols))
```
This is where the parallelizzation happens and local QR are created for each block. However to compute the inverse: $R^{-1}$, we call .compute() on $R$, pulling it back to the driver as a NumPy array, this introduces a bottleneck that breaks the laziness and centralizes $R$ on the driver.

---
• **Parallel / Dask approach (version 2)**

This version is equivalent to version 1 but replaces `.compute()` with the function: `.persist()`.
The function `.persist()` keeps $R$ distributed across the workers rather than pulling it to the driver.
This allows for improved results since the scheduler tracks dependencies and ensures $R$ is reused without recomputation.

Dask has da.linalg.qr, but it assumes the whole array is large and chunked regularly.
To get the final global R, you must combine all the Ri
That means at some point, the data has to come together into a single place (can’t keep it sharded).
So we bring the data to the driver since it is very small, this it optimizes the uses of np.linalg.qr, we are gathering the small stuff

---

• **Parallel / Dask approach (version 3)**

In this case instead of computing $R$ immediately it is left as a delayed object `dask.delayed()`, this means that no computation happens until `.compute()` is called at the end.
Afterwards instead of calculating the inverse immediately with NumPy this process is also delayed. This avoids pulling $R$ to the driver until the very end and allows Dask to schedule the inversion after $R$ is available in the graph, instead of serializing execution manually.
Ultimately also the final computation of $Q_da$ is lazy.
This fully delayed version allows the scheduler to optimize the entire pipeline together.

Cons:

Larger and more complex task graph (can become a scheduling overhead).

If you only need $R$ (or reuse $R$ multiple times), delaying everything might be inefficient compared to persist().

Execution time may fluctuate more since all steps (QR, stacking, inversion, multiplication) are chained into one big compute


In [ ]:
def indirect_serial(A, n_div):
    """
    Indirect TSQR (serial, NumPy).
    Splits A by rows into n_div blocks, computes local R_i via QR,
    reduces to global R by QR on the stacked R_i, then recovers Q = A R^{-1}.
    Returns (Q, R).
    """

    n_samp = A.shape[0]
    
    div_points = int(np.floor(n_samp/n_div))
    A_divided = []
    Ri = []
    
    A_divided = [A[div_points * i : div_points * (i + 1)] for i in range(n_div - 1)]    # Divide the A matrix into multiple chunks
    A_divided.append(A[(n_div - 1) * div_points:, :])   # In the case n_samp wasn't divisible by n_div

    Ri = [np.linalg.qr(Ai, mode="reduced")[1] for Ai in A_divided]
    R_stack = np.concatenate(Ri, axis = 0)
    _, R = np.linalg.qr(R_stack, mode="reduced")

    # Here you could also use the numpy function "np.linalg.inv(R)" function but the triangular decomposition grants more numerical stability
    I = np.eye(n_samp, dtype=A.dtype)
    Rinv = solve_triangular(R, I, lower=False)

    Q = A @ Rinv

    return Q, R


def compute_R(block):
    # np.linalg.qr with mode='r' gives just the R matrix
    R = np.linalg.qr(block, mode="r")
    return R


def indirect_parallel(X_da):
    """
    Indirect TSQR with Dask.
    Output:
        R    : final global triangular factor (n x n, NumPy array on driver)
        Q_da : Dask Array (m x n), representing Q = A R^{-1} (lazy)
    """

    n_cols = X_da.shape[1]

    # Parallel mapping of the QR blocks
    R_blocks = X_da.map_blocks(compute_R, dtype=X_da.dtype, chunks=(n_cols, n_cols))
    # Now R_blocks is a stack of n x n matrices (one per partition)
    # Its shape is (#chunks * n, n)

    # Bring all the blocks together to compute
    R_stack = R_blocks.compute()   # NumPy array, shape (p*n, n)

    # Small QR on driver to combine them into the final R
    R = np.linalg.qr(R_stack, mode="r")

    # Instead of materializing Q, compute a small R^{-1} (n x n).
    I = np.eye(n_cols, dtype=X_da.dtype)
    R_inv = solve_triangular(R, I, lower=False)  # stable

    # Broadcast Rinv to every chunk: Q = A @ R^{-1}
    Q_da = X_da @ R_inv   # still a Dask Array, lazy

    return Q_da, R      #Q_da because it is lazy, it is still a Dask array

def indirect_parallel_persisted(X_da):
    """
    Indirect TSQR with Dask.
    Output:
        R    : final global triangular factor (n x n, persisted)
        Q_da : Dask Array (m x n), representing Q = A R^{-1} (lazy)
    """

    n_cols = X_da.shape[1]
    R_blocks = X_da.map_blocks(compute_R, dtype=X_da.dtype, chunks=(n_cols, n_cols))

    # In this case instead of computing, persist the R blocks
    R_stack = R_blocks.persist()  
    _, R = np.linalg.qr(R_stack)

    I = np.eye(n_cols, dtype=X_da.dtype)
    R_inv = solve_triangular(R, I, lower=False)  # stable
    Q_da = X_da @ R_inv  

    return Q_da, R     



def indirect_parallel_delayed(X_da):
    """
    Indirect TSQR with Dask, delayed version.
    Output:
        R    : delayed object
        Q_da : Dask Array (m x n), representing Q = A R^{-1} (lazy)
    """

    n_cols = X_da.shape[1]
    R_blocks = X_da.map_blocks(compute_R, dtype=X_da.dtype, chunks=(n_cols, n_cols))


    # Convert blocks to delayed NumPy arrays, stack via delayed
    R_list = list(R_blocks.to_delayed().ravel())     # each is delayed np.ndarray (n x n)
    R_stack = dask.delayed(np.vstack)(R_list)        # delayed (p*n x n)
    
    R_delayed = dask.delayed(compute_R)(R_stack)      # delayed np.ndarray (n x n)


    I = np.eye(n_cols, dtype=X_da.dtype)
    # compute R^{-1} lazily
    R_inv_delayed = dask.delayed(solve_triangular)(R_delayed, I, lower=False)
    R_inv_da = dask.from_delayed(R_inv_delayed, shape=(n_cols, n_cols), dtype=X_da.dtype)

    # Broadcast multiply (keep Q lazy)
    Q_da = X_da @ R_inv_da

    return Q_da, R_delayed      #Q_da dask array, R delayed object


The following is an example of how to call the serial function in a local environment, an argument to pass is the number of partitions over which divide the dataset.

In [ ]:
%%time

Q, R = indirect_serial(data.data.values, 50)  # Divide in 50 partitions


The size for each chunk is : 0.0264192 Mb
CPU times: user 103 ms, sys: 82.4 ms, total: 186 ms
Wall time: 64.7 ms


In [ ]:
# not-delayed version
start = time.perf_counter()
Q_nd, R_nd = indirect_parallel(X_cached)  # returns Q_da, R (NumPy)
_ = R_nd  # already materialized
t_nd_R = time.perf_counter() - start



#Dask Dashboard: Big green block: compute_R. map stage of mapblock that returns the small Ri
# Tiny yellow finalize-hlg block - Dask housekeeping
"""Why you don’t see “reduce” or “broadcast” here
The reduce to the final 
R is done on the driver with NumPy:"""


t_nd_Q, val_nd = time_Q_apply(Q_nd, v)

"""Teal blocks around ~140–150 ms — blockwise-matmul-…
In the not-delayed variant, R^{-1} is a NumPy constant, so each matmul task deserializes it; you may notice slightly more per-task overhead compared to the delayed/Dask-array constant.
Purple block — reduction for the norm

After the matmul, da.linalg.norm(Qv) triggers a reduction:
"""


In [ ]:
# not-delayed persisted version
start = time.perf_counter()
Q_ndp, R_ndp = indirect_parallel_persisted(X_cached)  # returns Q_da, R (NumPy)
_ = R_ndp  # already materialized
t_nd_Rp = time.perf_counter() - start

t_po_Q, val_po = time_Q_apply(Q_ndp, v)


# better due to only calculating R once but with a cost
"""3. The subtlety

Persist = compute and cache now, but still return a Dask collection (with futures).

Compute = compute now and return the final NumPy array (collected to driver).

So:

If you persist inside your function and return R, you are indeed returning a Dask object backed by futures, not a NumPy matrix.

If you compute inside your function and return R, you’re returning a NumPy array, which is often what you want for the small triangular 
𝑅
R."""


'3. The subtlety\n\nPersist = compute and cache now, but still return a Dask collection (with futures).\n\nCompute = compute now and return the final NumPy array (collected to driver).\n\nSo:\n\nIf you persist inside your function and return R, you are indeed returning a Dask object backed by futures, not a NumPy matrix.\n\nIf you compute inside your function and return R, you’re returning a NumPy array, which is often what you want for the small triangular \n𝑅\nR.'

In [ ]:

# optimized not-delayed version
start = time.perf_counter()
Q_po, R_po = indirect_parallel_optimized(X_cached)  # returns Q_da (Dask), R_da (Dask)
_ = R_po  # already materialized
t_po_R = time.perf_counter() - start

t_po_Q, val_po = time_Q_apply(Q_po, v)


# I don't think this version makes sense
"""Purple (rechunk-merge): small graph-rewiring step. Dask is aligning the chunks of your broadcasted R_da with those of X_da. Because R_da is now itself a Dask Array (1 chunk of size 
𝑛
×
𝑛
n×n), it needs to merge graph metadata.

Big teal block (array) dominating the timeline:
This is your broadcasted matmul tasks: X_da @ Rinv_da.
Since R_inv was wrapped as a Dask Array (da.from_delayed or da.from_array), Dask sees a proper array-on-array multiply and generates blockwise matmul tasks.
→ That’s why you get this large contiguous band of teal tasks: each row-chunk of X_da multiplied with the single small (n,n) chunk of Rinv_da."""
"""Earlier, your “not-delayed” version showed slower Q @ v apply because:

You measured at small scales.

Serialization overhead per task looked big compared to the tiny math.

Wrapping in a Dask array made the graph smaller/cleaner → you saw ~0.06 s instead of ~0.5 s.

But at larger scales (big 
𝑚
m, many workers), the story flips:

Shipping one tiny NumPy array is cheap compared to all the matmuls.

Wrapping it as a Dask array adds pointless graph overhead.

So you can lose a bit of time overall."""


In [ ]:

# fully-delayed version
start = time.perf_counter()
Q_fd, R_fd = indirect_parallel_delayed(X_cached)  # returns Q_da (Dask), R_da (Dask)
_ = R_fd.compute()  # FORCE the same final R compute
t_fd_R = time.perf_counter() - start


t_fd_Q, val_fd = time_Q_apply(Q_fd, v)


"""Efficiency Implications

Computation of 
𝑅
R: essentially unchanged, still dominated by the map stage (compute_R).

Broadcast of 
𝑅
−
1
R
−1
: somewhat less efficient as a Dask Array, since it adds extra bookkeeping without reducing the numerical cost.

Norm benchmark (Q @ v): the visible red/yellow stages are expected; they confirm that your graph is carrying the computation fully through Dask.

because da.from_delayed introduces those extraction tasks.

Broadcast multiply still shows up as teal + red.

The dashboard is “busier” — more small tasks, more scheduler chatter — because everything, even tiny constants, was lifted into the Dask graph.

Efficiency interpretation

For small 
𝑛
n: making 
𝑅
R a Dask array (fully delayed) adds overhead without real benefit — the norm compute shows more yellow/red fragmentation than the optimized/persisted version.

For large distributed runs: it’s still correct, but NumPy constants (or scattered small arrays) are cheaper to handle than wrapping them in Dask.

That’s why your timings showed the fully delayed version wasn’t consistently faster"""


In [ ]:


print(f"R time — not-delayed: {t_nd_R:.3f}s | persisted: {t_po_Rp:.3f}s | optimized: {t_po_R:.3f}s | | full-delayed: {t_fd_R:.3f}s")


R time — not-delayed: 0.080s | persisted: 0.110s | optimized: 0.110s | partial: 0.081s | full-delayed: 0.073s


In [ ]:

print(f"Q apply — not-delayed: {t_nd_Q:.3f}s | partial: {t_po_Q:.3f}s | partial: {t_pd_Q:.3f}s | full-delayed: {t_fd_Q:.3f}s")


Q apply — not-delayed: 0.463s | partial: 0.054s | partial: 0.056s | full-delayed: 0.066s


In [10]:
N_WORKERS = 3
# Initialization of a distributed random matrix
m, n = int(1e7), 4
chunks = [m // N_WORKERS for _ in range(N_WORKERS-1)]
chunks.append(m - sum(chunks))
A = da.random.random((m, n), chunks=(chunks, n))

# Persist in memory to avoid recomputation
A = A.persist() 

print(f"Input matrix A: m = {A.shape[0]}, n = {A.shape[1]}")
print(f"The {len(A.chunks[0])} blocks are: {A.chunks[0]}")
print(f"Total size of A: {A.nbytes / 1e6} MB")
A


Input matrix A: m = 10000000, n = 4
The 3 blocks are: (3333333, 3333333, 3333334)
Total size of A: 320.0 MB


dask.array<random_sample, shape=(10000000, 4), dtype=float64, chunksize=(3333334, 4), chunktype=numpy.ndarray>

In [11]:
# QR decomposition

Q_delayed, R_delayed = indirect_parallel_delayed(A)  # Our implementation
Q_delayed_dask, R_delayed_dask = da.linalg.tsqr(A)    # Dask's implementation

print("-- OUR IMPLEMENTATION (QR) --")
%time Q, R = dask.compute(Q_delayed, R_delayed)

print("\n-- DASK'S IMPLEMENTATION (QR) --")
%time Q_dask, R_dask = dask.compute(Q_delayed_dask, R_delayed_dask)


-- OUR IMPLEMENTATION (QR) --
CPU times: user 1.06 s, sys: 1.37 s, total: 2.43 s
Wall time: 7.86 s

-- DASK'S IMPLEMENTATION (QR) --
CPU times: user 3.63 s, sys: 2.45 s, total: 6.07 s
Wall time: 8.82 s
